In [1]:
from cobra.io import read_sbml_model
from pathlib import Path
import os
import cobra

cobra.Configuration().solver = "gurobi"

# change working directory to context_specific_models folder
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

# create a dictionary with the filenames (keys) and models (values)
folder = Path().cwd()
files = [f for f in os.listdir(folder / 'cs_mods') if '.xml'in f]
models = {f:read_sbml_model(folder / 'cs_mods' / f ) for f in files}
k = list(models.keys())

# add model names as IDs
models_names = ['Human1', 'Schwan (1)', 'Schwan (2)', 'Melanocytes', 'Endothelial','Muscle','Fibroblasts',\
                'RPE','B cells','T + NK cells', 'Monocytes + Macrophages','Mast','Retina','RPE + Choroid',\
                'PR rod','Amacrine','Bipolar','PR cone', 'Horizontal', 'Müller','Ganglion']

for i in range(len(models)):
    models[k[i]].id = models_names[i]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


In [2]:
import pandas as pd

mod = models[k[0]].copy()

# add back any lost exchange reactions
def add_all_EX_rxns(basis_model,model):
    EX_rxns = [r for r in basis_model.reactions if len(r.products)==0]
    EX_rxns_model = [r.id for r in model.reactions if len(r.products)==0]
    missing_rxns = [r.copy() for r in EX_rxns if r.id not in EX_rxns_model]
    model.add_reactions(missing_rxns)
    return model

# add ATP demand reactions, lost exchange reactions to all models
for k_i in k:
    models[k_i].add_reactions([mod.reactions.get_by_id('MAR03964').copy()])
    models[k_i] = add_all_EX_rxns(mod,models[k_i])
    for r in [r for r in models[k_i].reactions if len(r.products)==0]: r.bounds = (0,1000)     # close lower bound for all exchange reactions (metabolites can leave the system but not enter)

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpr_cqk2yn.lp
Reading time = 0.12 seconds
: 8369 rows, 26140 columns, 111668 nonzeros


Ignoring reaction 'MAR03964' since it already exists.


In [3]:
# make RPE-PR model

# copy RPE, PR, and full Human1 models
mod = models['Human-GEM.xml'].copy()
mod_RPE = models['model_06_MeanExpression_RPE__VoigtEtAl2019_.xml'].copy()
mod_PR = models['model_13_MeanExpression_RodPhotoreceptors__LiangEtAl_2019_.xml'].copy()

def remove_compartment(model, compartment):
# remove compartment from model
    rxns_n = [r.id for r in model.reactions if compartment in r.compartments]
    mets_n = [m for m in model.metabolites if compartment in m.compartment]
    model.remove_reactions(rxns_n)
    model.remove_metabolites(mets_n)
    return model

# remove nucleus from PR model
mod_PR = remove_compartment(mod_PR, 'n')

def add_id_suffix(model, suffix):
    for r in model.reactions:
        r.id = r.id + suffix
    for m in model.metabolites:
        m.id = m.id + suffix
        m.compartment = m.compartment + suffix
    return model
# add '_PR' to PR model IDs and '_RPE' to RPE model IDs
mod_PR = add_id_suffix(mod_PR, '_PR')
mod_RPE = add_id_suffix(mod_RPE, '_RPE')

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpf1ljlojh.lp
Reading time = 0.11 seconds
: 8369 rows, 26140 columns, 111668 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpwtm4fy8y.lp
Reading time = 0.06 seconds
: 5517 rows, 14194 columns, 54382 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp1va61ike.lp
Reading time = 0.05 seconds
: 5013 rows, 12490 columns, 46478 nonzeros


C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [4]:
# add RPE-PR interface reactions to RPE model (rxn_RPE-PR = RPE <--> RPE-PR interface)
for rxns in  [r for r in mod_RPE.reactions if 'e_RPE' in r.reaction]:     
    rxn = rxns.copy()
    rxn.id = rxn.id.replace('_RPE','_RPE-PR')
    for p in rxn.products:
        if 'e_RPE' in p.id:
            p.id = p.id.replace('e_RPE','e_RPE-PR')
            p.compartment = 'e_RPE-PR'
    for r in rxn.reactants:
        if 'e_RPE' in r.id:
            r.id = r.id.replace('e_RPE','e_RPE-PR')
            r.compartment = 'e_RPE-PR'
    mod_RPE.add_reactions([rxn])
    
# add RPE-PR interface reactions to PR model (rxn_PR-RPE = PR <--> RPE-PR interface)
for rxn in [r for r in mod_PR.reactions if 'e_PR' in r.reaction]:
    rxn.id = rxn.id.replace('_PR','_PR-RPE')
    for p in rxn.products:
        if 'e_PR' in p.id:
            p.id = p.id.replace('e_PR','e_RPE-PR')
            p.compartment = 'e_RPE-PR'
    for r in rxn.reactants:
        if 'e_PR' in r.id:
            r.id = r.id.replace('e_PR','e_RPE-PR')
            r.compartment = 'e_RPE-PR'

In [5]:
# fuse RPE and PR model
from cobra import Model
mod_RPE_PR = Model('RPE-PR model')
mod_RPE_PR.add_reactions(mod_RPE.reactions)
mod_RPE_PR.add_reactions(mod_PR.reactions)

In [6]:
# list reaction IDs to be deleted (because their reactions are duplicated)
import pandas as pd
df = pd.DataFrame([[r.id, r.reaction] for r in mod_RPE_PR.reactions \
                   if r.compartments == {'e_RPE-PR'}], columns=['id','reaction'])
l  = list(df['id'][df['reaction'].duplicated()])

# delete duplicated reactions and change reaction IDs (suffix: _eRPE-PR) 
# of the reactions that only involve the e_RPE-PR compartment (interface)
for r in [r for r in mod_RPE_PR.reactions if r.compartments=={'e_RPE-PR'}]: 
    if r.id in l:
        mod_RPE_PR.remove_reactions(r.id)
    elif '_RPE-PR' in r.id:
        r.id = r.id.replace('_RPE-PR','_eRPE-PR')
    elif '_PR-RPE' in r.id:
        r.id = r.id.replace('_PR-RPE','_eRPE-PR')
        ,
def fix_compartment_dict(model):
    new_compartment_dict = model.compartments.copy()
    for k in list(new_compartment_dict.keys()):
        new_compartment_dict[k] = k 
    model.compartments = new_compartment_dict
    return model
fix_compartment_dict(mod_RPE_PR)

# close lower bounds of exchange reactions into RPE-PR interface space (nothing allowed in from outside)
rxns = [r for r in mod_RPE_PR.reactions if len(r.products)==0 if r.lower_bound<0 if 'eRPE-PR' in r.id]
for rxn in rxns: rxn.lower_bound = 0

C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\model.py:778: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [7]:
from cobra.io import write_sbml_model
write_sbml_model(mod_RPE_PR, "mod_RPE-PR.xml")

In [8]:
# set bounds Exchange reactions (lower bounds -1000, upper bounds based on file info)  

# import excel file with info on exchange upper bounds 
# file contains 'id' col with IDs and an 'upper_bound' col with upper bounds to modify
bounds = pd.read_excel(folder / 'rxn_bounds' / 'R3D301_EX_rxns.xlsx')
bounds = bounds[bounds['upper_bound']>0] # select entries with non-zero upper bounds

for i in range(len(bounds)): 
    models[k_i].reactions.get_by_id(bounds['id'][i]).bounds = (-bounds['upper_bound'][i],1000)   

In [9]:
# open RPE exchange reactions
for i in range(len(bounds)): 
    mod_RPE_PR.reactions.get_by_id(bounds['id'][i]+'_RPE').bounds = (-bounds['upper_bound'][i],1000)   

In [10]:
# ATP demand reactions RPE
mod_RPE_PR.reactions.get_by_id('MAR03964_RPE')

Reaction identifier,MAR03964_RPE
Name,ATP phosphohydrolase
Memory address,0x1be5612d6c0
Stoichiometry,MAM01371c_RPE + MAM02040c_RPE --> MAM01285c_RPE + MAM02039c_RPE + MAM02751c_RPE ATP + H2O --> ADP + H+ + Pi
GPR,
Lower bound,0.0
Upper bound,1000.0


In [11]:
# ATP demand reactions PR
mod_RPE_PR.reactions.get_by_id('MAR03964_PR')

Reaction identifier,MAR03964_PR
Name,ATP phosphohydrolase
Memory address,0x1be5b018cd0
Stoichiometry,MAM01371c_PR + MAM02040c_PR --> MAM01285c_PR + MAM02039c_PR + MAM02751c_PR ATP + H2O --> ADP + H+ + Pi
GPR,
Lower bound,0.0
Upper bound,1000.0


In [12]:
# glucose exchange reaction RPE
mod_RPE_PR.reactions.get_by_id('MAR09034_RPE')

Reaction identifier,MAR09034_RPE
Name,
Memory address,0x1be55de3640
Stoichiometry,MAM01965e_RPE <=> glucose <=>
GPR,
Lower bound,-40.0
Upper bound,1000


In [13]:
# OBJECTIVE 
# set maximizing the ATP demand in PR as objective
mod_RPE_PR.objective = mod_RPE_PR.reactions.get_by_id('MAR03964_PR').flux_expression
solution = mod_RPE_PR.optimize()
print(solution)

<Solution 83.215 at 0x1be41c11e40>


In [40]:
def get_vmh_id(reaction):
    vmh_id = ''
    if 'vmhreaction' in reaction.annotation.keys(): 
        vmh_id = reaction.annotation['vmhreaction']
    return  vmh_id
    
df= pd.DataFrame([[get_vmh_id(r),r.name,r.subsystem,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
              for r in mod_RPE_PR.reactions],\
                 index = [r.id  for r in mod_RPE_PR.reactions],
             columns=['vmh id', 'name', 'subsystem', 'reaction (id)', 'reaction (name)'])

,vmh id,name,subsystem,reaction (id),reaction (name)
MAR03905_RPE,ALCD2if,ethanol:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,MAM01796c_RPE + MAM02552c_RPE --> MAM01249c_RP...,ethanol + NAD+ --> acetaldehyde + H+ + NADH
MAR03907_RPE,ALCD2yf,Ethanol:NADP+ oxidoreductase,Glycolysis / Gluconeogenesis,MAM01796c_RPE + MAM02554c_RPE --> MAM01249c_RP...,ethanol + NADP+ --> acetaldehyde + H+ + NADPH
MAR04281_RPE,r0173,(S)-Lactate:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,MAM02039x_RPE + MAM02553x_RPE + MAM02819x_RPE ...,H+ + NADH + pyruvate <=> L-lactate + NAD+
MAR04388_RPE,LDH_L,(S)-Lactate:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,MAM02039c_RPE + MAM02553c_RPE + MAM02819c_RPE ...,H+ + NADH + pyruvate <=> L-lactate + NAD+
MAR04283_RPE,ALDD2y,Acetaldehyde:NADP+ oxidoreductase,Glycolysis / Gluconeogenesis,MAM01249c_RPE + MAM02040c_RPE + MAM02554c_RPE ...,acetaldehyde + H2O + NADP+ --> acetate + 2.0 H...
...,...,...,...,...,...
MAR10065_PR,,Cofactors and vitamins pool for biomass reaction,Artificial reactions,0.0345 MAM00291c_PR + 0.0172 MAM00427c_PR + 0....,0.0345 11-cis-retinol + 0.0172 18-hydroxy-all-...
MAR10067_PR,,,Inositol phosphate metabolism,MAM00521c_PR + MAM02040c_PR --> MAM00525c_PR +...,"1D-myo-inositol-1,3,4,5,6-pentakisphosphate + ..."
MAR10068_PR,,,Inositol phosphate metabolism,MAM00525c_PR + MAM02040c_PR --> MAM01718c_PR +...,"1D-myo-inositol-1,4,5,6-tetrakisphosphate + H2..."
MAR13083_PR,,,Metabolism of other amino acids,MAM02039c_PR + MAM02157c_PR + MAM02553c_PR + M...,H+ + hypotaurine + NADH + O2 --> H2O + NAD+ + ...


In [ ]:
objectives = {mod_RPE_PR.reactions.get_by_id('MAR03964_PR'): 2, mod_RPE_PR.reactions.get_by_id('MAR03964_RPE'): 1}

from cobra.flux_analysis import flux_variability_analysis
mod_RPE_PR.objective = 'MAR03964_PR'
mod_RPE_PR.objective.expression
fva = flux_variability_analysis(mod_RPE_PR, loopless=True)

In [46]:
pd.merge(df, solution.to_frame(), left_index=True, right_index=True)

,vmh id,name,subsystem,reaction (id),reaction (name),fluxes,reduced_costs
MAR03905_RPE,ALCD2if,ethanol:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,MAM01796c_RPE + MAM02552c_RPE --> MAM01249c_RP...,ethanol + NAD+ --> acetaldehyde + H+ + NADH,0.000000,0.000000
MAR03907_RPE,ALCD2yf,Ethanol:NADP+ oxidoreductase,Glycolysis / Gluconeogenesis,MAM01796c_RPE + MAM02554c_RPE --> MAM01249c_RP...,ethanol + NADP+ --> acetaldehyde + H+ + NADPH,0.000000,0.000000
MAR04281_RPE,r0173,(S)-Lactate:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,MAM02039x_RPE + MAM02553x_RPE + MAM02819x_RPE ...,H+ + NADH + pyruvate <=> L-lactate + NAD+,0.000000,0.000000
MAR04388_RPE,LDH_L,(S)-Lactate:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,MAM02039c_RPE + MAM02553c_RPE + MAM02819c_RPE ...,H+ + NADH + pyruvate <=> L-lactate + NAD+,73.290256,0.000000
MAR04283_RPE,ALDD2y,Acetaldehyde:NADP+ oxidoreductase,Glycolysis / Gluconeogenesis,MAM01249c_RPE + MAM02040c_RPE + MAM02554c_RPE ...,acetaldehyde + H2O + NADP+ --> acetate + 2.0 H...,0.000000,0.000000
...,...,...,...,...,...,...,...
MAR10065_PR,,Cofactors and vitamins pool for biomass reaction,Artificial reactions,0.0345 MAM00291c_PR + 0.0172 MAM00427c_PR + 0....,0.0345 11-cis-retinol + 0.0172 18-hydroxy-all-...,0.000000,-5.382796
MAR10067_PR,,,Inositol phosphate metabolism,MAM00521c_PR + MAM02040c_PR --> MAM00525c_PR +...,"1D-myo-inositol-1,3,4,5,6-pentakisphosphate + ...",0.000000,0.000000
MAR10068_PR,,,Inositol phosphate metabolism,MAM00525c_PR + MAM02040c_PR --> MAM01718c_PR +...,"1D-myo-inositol-1,4,5,6-tetrakisphosphate + H2...",0.000000,0.000000
MAR13083_PR,,,Metabolism of other amino acids,MAM02039c_PR + MAM02157c_PR + MAM02553c_PR + M...,H+ + hypotaurine + NADH + O2 --> H2O + NAD+ + ...,0.000000,-12.971235


In [30]:
solution.to_frame()

,fluxes,reduced_costs
MAR03905_RPE,0.000000,0.000000
MAR03907_RPE,0.000000,0.000000
MAR04281_RPE,0.000000,0.000000
MAR04388_RPE,73.290256,0.000000
MAR04283_RPE,0.000000,0.000000
...,...,...
MAR10065_PR,0.000000,-5.382796
MAR10067_PR,0.000000,0.000000
MAR10068_PR,0.000000,0.000000
MAR13083_PR,0.000000,-12.971235


In [14]:
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
solution_loopless = loopless_solution(mod_RPE_PR)

import pandas
df = pandas.DataFrame(dict(loopless=solution_loopless.fluxes, nominal=solution.fluxes))

In [26]:
solution_loopless

,fluxes,reduced_costs
MAR03905_RPE,0.000000,-1.000000
MAR03907_RPE,0.000000,-1.000000
MAR04281_RPE,0.000000,-1.000000
MAR04388_RPE,1.308654,-1.000000
MAR04283_RPE,0.000000,35.795841
...,...,...
MAR10065_PR,0.000000,197.058682
MAR10067_PR,0.000000,-1.000000
MAR10068_PR,0.000000,-1.000000
MAR13083_PR,0.000000,850.249990


In [25]:
df

,loopless,nominal
MAR03905_RPE,0.000000,0.000000
MAR03907_RPE,0.000000,0.000000
MAR04281_RPE,0.000000,0.000000
MAR04388_RPE,1.308654,73.290256
MAR04283_RPE,0.000000,0.000000
...,...,...
MAR10065_PR,0.000000,0.000000
MAR10067_PR,0.000000,0.000000
MAR10068_PR,0.000000,0.000000
MAR13083_PR,0.000000,0.000000


In [15]:
# 
#with mod_RPE_PR:
#    add_loopless(mod_RPE_PR)
#    solution_add_loopless = mod_RPE_PR.optimize()
#print("loopless solution: status = " + solution_add_loopless.status)
#print("loopless solution flux: v3 = %.1f" % solution_add_loopless.fluxes["v3"])
# solution_add_loopless.fluxes

In [23]:
mod_RPE_PR.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
MAM01965e_RPE,MAR09034_RPE,40,6,83.05%
MAM02184e_RPE,MAR09039_RPE,0.5077,6,1.05%
MAM02360e_RPE,MAR09040_RPE,0.6538,6,1.36%
MAM03135e_RPE,MAR09046_RPE,1.179,5,2.04%
MAM01975e_RPE,MAR09063_RPE,2.651,5,4.59%
MAM01365e_RPE,MAR09066_RPE,0.2615,6,0.54%
MAM01986e_RPE,MAR09067_RPE,0.7031,2,0.49%
MAM01974e_RPE,MAR09071_RPE,1.338,5,2.32%
MAM02658e_RPE,MAR09087_RPE,0.9333,5,1.61%
MAM01587e_RPE,MAR09286_RPE,0.6564,6,1.36%


In [20]:
solution_loopless

,fluxes,reduced_costs
MAR03905_RPE,0.000000,-1.000000
MAR03907_RPE,0.000000,-1.000000
MAR04281_RPE,0.000000,-1.000000
MAR04388_RPE,1.308654,-1.000000
MAR04283_RPE,0.000000,35.795841
...,...,...
MAR10065_PR,0.000000,197.058682
MAR10067_PR,0.000000,-1.000000
MAR10068_PR,0.000000,-1.000000
MAR13083_PR,0.000000,850.249990


In [16]:
from cobra.flux_analysis import pfba
solution_pfba = pfba(mod_RPE_PR)
print("parsimonious solution: status = " + solution_pfba.status)

parsimonious solution: status = optimal


KeyError: 'v3'

In [18]:
solution_pfba

,fluxes,reduced_costs
MAR03905_RPE,0.000000,-2.000000
MAR03907_RPE,0.000000,1.733333
MAR04281_RPE,0.000000,0.266667
MAR04388_RPE,-2.232513,2.000000
MAR04283_RPE,0.000000,6.733333
...,...,...
MAR10065_PR,0.000000,824.559227
MAR10067_PR,0.000000,-2.000000
MAR10068_PR,0.000000,-2.000000
MAR13083_PR,0.000000,1683.600226


In [19]:
solution_pfba.fluxes

MAR03905_RPE     0.000000
MAR03907_RPE     0.000000
MAR04281_RPE     0.000000
MAR04388_RPE    -2.232513
MAR04283_RPE     0.000000
                  ...    
MAR10065_PR      0.000000
MAR10067_PR      0.000000
MAR10068_PR      0.000000
MAR13083_PR      0.000000
MAR03964_PR     83.215128
Name: fluxes, Length: 14282, dtype: float64

In [24]:
solution_pfba.to_frame()

,fluxes,reduced_costs
MAR03905_RPE,0.000000,-2.000000
MAR03907_RPE,0.000000,1.733333
MAR04281_RPE,0.000000,0.266667
MAR04388_RPE,-2.232513,2.000000
MAR04283_RPE,0.000000,6.733333
...,...,...
MAR10065_PR,0.000000,824.559227
MAR10067_PR,0.000000,-2.000000
MAR10068_PR,0.000000,-2.000000
MAR13083_PR,0.000000,1683.600226


In [ ]:
s = mod_RPE_PR.summary(fva=0.95)

In [ ]:
solution.to_frame()

In [ ]:
s.to_frame()

In [ ]:
mod_RPE_PR.reactions.get_by_id('MAR09034_RPE')

In [ ]:
# with mod_RPE_PR:
#    add_loopless(mod_RPE_PR)
#    try: solution = mod_RPE_PR.optimize()
#    except: print('model is infeasible')

In [ ]:
# FVA
#from cobra.flux_analysis import flux_variability_analysis
#flux_variability_analysis(mod_RPE_PR,loopless=True)

In [ ]:
# list all exchange and demand reactions matching Human1 and VMH IDs
def get_vmh_id(reaction):
    if 'vmhreaction' in reaction.annotation.keys(): vmh_id = reaction.annotation['vmhreaction']
    else: vmh_id = ''
    return vmh_id
df_EX_rxns = pd.DataFrame([[r.id,get_vmh_id(r),r.name,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
              for r in mod.reactions if len(r.products) == 0],columns=['id','vmh id','name','reaction','reaction (names)'])
df_EX_rxns

In [ ]:
# ATP metabolites
ATP_mets = [[met.id,met.name] for met in mod_RPE_PR.metabolites if met.name=='ATP']
pd.DataFrame(ATP_mets,columns=['ID','name'])

# ATP reactions
ATP_rxns = [[r.id,get_vmh_id(r),r.name,r.subsystem,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
            for r in mod_RPE_PR.reactions if 'MAM01371' in r.reaction]

pd.DataFrame(ATP_rxns,columns=['id','vmh id', 'name', 'subsystem', 'reaction (id)', 'reaction (name)'])

In [ ]:
# list all exchange and demand reactions
df_EX_rxns_human1 = pd.DataFrame([[r.id,get_vmh_id(r),r.name,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
              for r in mod.reactions if len(r.products)==0],columns=['id','vmh id','name','reaction','reaction (names)'])
df_EX_rxns.to_clipboard(excel=True, sep=None)
df_EX_rxns

In [ ]:
# match VMH_IDs from bounds file to Human1 IDs to create 'R3D301_EX_rxns_RPE-PR.xlsx'

# import excel file with info on bounds
bounds = pd.read_excel(folder / 'rxn_bounds' / 'files PL' / 'R3D301_EX_rxns.xlsx')

# vmh_id lookup table + look up RPE rxn
df_vmh_id = pd.DataFrame([[get_vmh_id(r),r.id,r.id+'_RPE'] for r in mod.reactions if 'EX_' in get_vmh_id(r)],columns=['vmh_id','human1_id','id'])
merged_df = df_vmh_id.merge(bounds, how = 'right', on = ['vmh_id'])
merged_df.to_clipboard(excel=True, sep=None)
merged_df

In [ ]:
# biomass reaction
mod.reactions.get_by_id('MAR04413') 

pd.DataFrame([[r.id,r.name,r.reaction,r.reaction,r.build_reaction_string(use_metabolite_names = True)]\
              for r in mod_RPE_PR.reactions if 'biomass' in r.name])

In [ ]:
mod.reactions.get_by_id('MAR13082') 